# A1

Using a time series model, is there a seasonal trend of when revenue increases and decreases? A decrease in revenue could possibly indicate customer churn and from there analayze if there is a seasonal trend indicating if there is a pattern in customer churning. 

# A2

The goal is to build a time series model where we can predict future revenue predictions, observe seasonality trends and patterns. A decrease in revenue can indicate customer churn.

# B

-Data does not contain anomalies
-Model parameters and error term is constant
-Historic timepoints dictate behavior of present timepoints which might not hold in stressed market data conditions 
-Time series is stationary

# C1

In [ ]:
# import a couple of libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from scipy.optimize import minimize
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from tqdm import tqdm_notebook
from itertools import product
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)/y_true))*100
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
# import data
df = pd.read_csv("teleco_time_series .csv")
# look at the head
df.head(10)

In [ ]:
df.info()

In [ ]:
df.duplicated()

In [ ]:
df.isnull().any()

In [ ]:
plt.figure(figsize=(17, 8))
plt.plot(df.Revenue)
plt.title('Company Revenue last two years')
plt.xlabel('day')
plt.ylabel('Revenue (mil)')
plt.grid(False)
plt.show()

# C2

The time step format of this dataset is measured in days, it can be seen that the data type for day is in integers. It also shows that there are no gaps in the dataset. The length of the sequence is 731 data points. Also the dataset shows that there are no duplicates.

# C3

As seen on the 'Company Revenue last two years' plot, the can be see that it is non-stationary as the plot has an upward trend.

# C4

#1) understand the dataframe using the .info() function
#2) convert time column into datetime format if column is stored as string
#3) check for trend, seasonality, and noise

# C5

In [ ]:
import math
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
Revenue = df['Revenue']
Revenue

In [ ]:
lnrevenue = np.log(Revenue)
lnrevenue
plt.plot(lnrevenue)
plt.show()

In [ ]:
acf_1 = acf(Revenue)[1:20]
test_df = pd.DataFrame([acf_1]).T
test_df.columns = ['Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
plt.show()

In [ ]:
pacf_1 = pacf(Revenue)[1:20]
test_df = pd.DataFrame([pacf_1]).T
test_df.columns = ['Partial Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
plt.show()

In [ ]:
result = ts.adfuller(Revenue, 1)
result
Revenue_diff = Revenue-Revenue.shift()
diff=Revenue_diff.dropna()
acf_1_diff = acf(diff)[1:20]
test_df = pd.DataFrame([acf_1_diff]).T
test_df.columns = ['First Diff Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
pacf_1_diff = pacf(diff)[1:20]
plt.plot(pacf_1_diff)
plt.show()

In [ ]:
test_df = pd.DataFrame([pacf_1_diff]).T
test_df.columns = ['First Diff Partial Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
plt.show()

revenue_matrix = Revenue.as_matrix(columns=None)
model = ARIMA(revenue_matrix, order=(0,1,0))
model.fit = model.fit(disp=0)
print(model_fit.summary())
predictions=model_fit.predict(122, 127, typ='levels')
predictions
predictionsadjusted= np.exp(predictions)
predictionsadjusted
plt.plot(predictionsadjusted)
plt.title('Forecasted Revenue')

In [ ]:
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(Revenue)

df.index = df.index.to_period('M')

In [ ]:
model = ARIMA(Revenue, order=(100,1,0))
model_fit=model.fit()

In [ ]:
print(model_fit.summary())

# D1

# D3

# D4

# D5

# E1

# E2

# E3

# F

# G

# H

# I